<a href="https://colab.research.google.com/github/saitarun18/DNGP_096/blob/main/DN_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import seaborn as sns
import numpy as np

##Main Code

In [ ]:

class EdgeEnvironment:


    def __init__(self,deviceBattery,deviceMaxBattery,ECPower,ECStorage,ECMaxStorage,deviceData): 

      self.deviceMaxBattery=deviceMaxBattery
      self.deviceBattery=deviceBattery
      self.ECMaxStorage=ECMaxStorage
      self.ECPower=ECPower
      self.ECStorage=ECStorage
      self.deviceData=deviceData
      self.deviceDict = {}
      self.edgeDict= {}
      self.visitedEdge = [False for i in self.ECPower]
      self.powerUsage=[[inf for i in range(len(deviceMaxBattery))] for j in range(len(ECMaxStorage))]


    def extractInput(self,inputArray):
      ##inputArray = device-edge-power
      for elem in inputArray:
        if elem[1] in self.edgeDict:
            self.edgeDict[elem[1]].append(elem[0])
        else:
            self.edgeDict[elem[1]] = [elem[0]]

      for elem in inputArray:
        if elem[0] in self.deviceDict:
            self.deviceDict[elem[0]].append(elem[1])
        else:
            self.deviceDict[elem[0]] = [elem[1]]

      for i,j,power in inputArray:
        self.powerUsage[j][i]=power

    def startAllocation(self):
      #while sum(self.visitedEdge)<len(self.ECPower):
      for _ in range(len(self.ECPower)):
        edgeComputer = self.getEdgeComputer()
        self.allocateEdgeComputer(edgeComputer)

    def getEdgeComputer(self): 
        costArray = []

        for i in range(len(self.ECPower)):
          sum=0
          for j in self.edgeDict[i]:
            sum += self.deviceData[j]
          if not sum or self.visitedEdge[i]:
            self.visitedEdge[i] = True
            costArray.append(float('inf'))
          else:
            costArray.append(self.ECPower[i]/sum)

        miny = min(costArray)

        index = costArray.index(miny)
        self.visitedEdge[index] = True

        #print(index)
        return index

    def allocateEdgeComputer(self,edgeComputer):
      connectedDevices = self.edgeDict[edgeComputer]
      
      ##Sorting devices connected to the given Edge Computer based on power needed to send through that connection per Unit data
      connectedDevices.sort(key = lambda x: self.powerUsage[edgeComputer][x])

      ##For each device in the sorted connected Devices we will allocate it using greedy algorithm
      for device in connectedDevices:
        ##This method allocateDeviceToEC return True if data is allocated else False
        self.allocateDeviceToEC(device,edgeComputer)

    def allocateDeviceToEC(self,device,edgeComputer): ##Will allocate data to edge computer from device using greedy
      ## Here sending Data is the min of storage available at EC and data needed to send
      sendingData = min(self.deviceData[device],self.ECMaxStorage[edgeComputer]-self.ECStorage[edgeComputer])

      ##Here power Needed is the min of power available at device and power needed to send sending Data
      powerNeeded = min(self.deviceBattery[device],self.powerUsage[edgeComputer][device]*sendingData)

      ##If power needed is 0 which is not possible we will abort by return False as no allocation done
      if not self.powerUsage[edgeComputer][device]:
        return 

      ##Here sendingData is changed to according to power
      sendingData = powerNeeded/self.powerUsage[edgeComputer][device]
      ##From here sendingData and powerNeeded will obey all the constraints

      ##Updating all the variables
      self.ECStorage[edgeComputer] += sendingData
      self.deviceBattery[device] -= powerNeeded
      self.deviceData[device] -= sendingData
      #print(f'From device {device} to edge Computer {edgeComputer} data Transferred is {sendingData}')
      ## if sended Data
      return


##Defined data range

In [ ]:
minEdgeDevices,maxEdgeDevices = 50,100
minDevices,maxDevices = 150,400
minDeviceBattery,maxDeviceBattery = 900,1000
minEdgePower,maxEdgePower = 8000,10000
minDeviceStorage,maxDeviceStorage = 10,100
minEdgeStorage,maxEdgeStorage = 200000,450000
minTransmissionPower,maxTransmissionPower = 1,2
inf = float('inf')

##Sample code for graphs and Testing

In [ ]:
deviceStorage = [60, 80, 75, 70, 68, 86]

In [ ]:
m = random.randint(minEdgeDevices,maxEdgeDevices) ##No of edge devices
n = random.randint(minDevices,maxDevices) ##No of devices

deviceMaxBattery,deviceData,ECPower,ECMaxStorage,ECStorage = [],[],[],[],[0 for i in range(m)]

for i in range(m):
  ECPower.append(random.randint(minEdgePower,maxEdgePower))
  ECMaxStorage.append(random.randint(minEdgeStorage,maxEdgeStorage))

for i in range(n):
  deviceMaxBattery.append(random.randint(minDeviceBattery,maxDeviceBattery))
  deviceData.append(random.randint(minDeviceStorage,maxDeviceStorage))
deviceBattery=deviceMaxBattery[::]

In [ ]:
inputArray = [] ##device-edge-power
for i in range(n):
  numberOfEdges = random.randint(1,m-1)
  connectedEdges = random.sample(range(m),numberOfEdges)
  for j in connectedEdges:
    inputArray.append([i,j,float(random.randint(minTransmissionPower,maxTransmissionPower))])

In [ ]:
print(f'no of edges: {m}')
print(f'no of devices: {n}')
print('deviceMaxBatter:',deviceMaxBattery)
print('deviceBattery:',deviceBattery)
print('deviceData:',deviceData)
print('ECPower:',ECPower)
print('ECStorage:',ECStorage)
print('ECMaxStorage:',ECMaxStorage)
print(inputArray)

In [ ]:
obj = EdgeEnvironment(deviceBattery,deviceMaxBattery,ECPower,ECStorage,ECMaxStorage,deviceData)
obj.extractInput(inputArray)

In [ ]:
AllocationData = deviceData[:]

In [ ]:
obj.startAllocation()

##Test Cases

In [ ]:
noOfValues = 300
x1,y1,x3,y3,x4,y4 = [],[],[],[],[],[]
for iii in range(noOfValues):
  m = random.randint(minEdgeDevices,maxEdgeDevices) ##No of edge devices
  n = random.randint(minDevices,maxDevices) ##No of devices

  deviceMaxBattery,deviceData,ECPower,ECMaxStorage,ECStorage = [],[],[],[],[0 for i in range(m)]

  for i in range(m):
    ECPower.append(random.randint(minEdgePower,maxEdgePower))
    ECMaxStorage.append(random.randint(minEdgeStorage,maxEdgeStorage))

  for i in range(n):
    deviceMaxBattery.append(random.uniform(minDeviceBattery,maxDeviceBattery))
    deviceData.append(random.randint(minDeviceStorage,maxDeviceStorage))
  deviceBattery=deviceMaxBattery[::]
  AllocationData = deviceData[:]
  inputArray = [] ##device-edge-power
  for i in range(n):
    numberOfEdges = random.randint(1,m-1)
    connectedEdges = random.sample(range(m),numberOfEdges)
    for j in connectedEdges:
      inputArray.append([i,j,float(random.randint(minTransmissionPower,maxTransmissionPower))])
  obj = EdgeEnvironment(deviceBattery,deviceMaxBattery,ECPower,ECStorage,ECMaxStorage,deviceData)
  obj.extractInput(inputArray)  
  AllocationData = deviceData[:]
  obj.startAllocation()
  x,y = Graph1()
  x1.append(x)
  y1.append(y)

  x,y = Graph3()
  x3.append(x)
  y3.append(y)

  x,y = Graph4()
  x4.append(x)
  y4.append(y)

  print(iii+1,end=' ')

print()


In [ ]:
print(x1)
print(y1)

In [ ]:
print(x2)
print(y2)

In [ ]:
print(x3)
print(y3)

In [ ]:
print(x4)
print(y4)

## Graph - 1

In [ ]:
def Graph1():
  powerConsumedByDevices = np.subtract(obj.deviceMaxBattery, obj.deviceBattery)
  dataSentByDevices = np.subtract(obj.ECMaxStorage, obj.ECStorage)
  
  return (sum(dataSentByDevices),sum(powerConsumedByDevices))

In [ ]:
g1x = [1766357.8132679758, 1537572.9824533144, 1747055.6188805948, 1057855.2142292093, 1012463.190920355, 1779876.0513704885, 1665026.5889087839, 1733775.3060132693, 1830261.6292181802, 1733286.1421435354, 1928505.715245211, 1158185.5762530232, 1821320.0302981401, 1254921.4499200638, 1442657.3670760782, 1911030.4027949984, 1510769.9298335137, 1180359.8279166603, 1857396.3250698466, 1924031.9280750523, 1378846.8216817447, 1691835.3128196457, 1520190.0982180014, 1497378.7536213421, 1278716.0743449207, 1900617.976176619, 1402989.9470415905, 1874931.2871586136, 1027689.4423867201, 1299515.3603396094, 1096517.0525017856, 1935191.072141861, 1709627.9842155068, 1119826.8199217205, 1215165.9847202138, 1250519.4854492843, 1065240.2512118458, 1043050.7245134466, 1286456.5712690665, 1155492.7616311302, 1252739.7917654512, 1175104.535945504, 1146420.4482521429, 1052575.4545894102, 1396510.9857859584, 1640431.4801805625, 1345025.8045618285, 1584478.974603421, 1115733.1438590274, 1578068.089498487, 1734486.5533344324, 1363701.6641321415, 1509259.0253042195, 1432826.9449693155, 1913452.4425098838, 1496547.610730024, 1126675.8465190586, 1311169.457859443, 1229837.8309263377, 1422516.1677278704, 1131330.774225748, 1711453.2547979003, 1551450.2079009286, 1135421.8745096885, 1108607.244761684, 1602129.053199319, 1541681.2510060086, 1326264.1462642776, 1825406.6434300814, 1987069.3082223616, 1369178.992406445, 1505492.5351470243, 1459389.8964827019, 1983612.1021482951, 1882856.8722848746, 1504209.6346939767, 1092242.0440395079, 1936106.734119744, 1880043.7440686063, 1856813.2754515237, 1118415.7569940647, 1052255.1637912935, 1841968.2779959384, 1919617.3030819688, 1749441.2225518245, 1271245.5709352866, 1016332.2078352435, 1451153.058943783, 1575697.4795142892, 1435863.5153869656, 1262896.3864885499, 1391464.1574297012, 1167639.95825389, 1023798.6262804937, 1841194.257251824, 1989425.7359981367, 1977219.6782805556, 1528763.0305654649, 1148853.4703715297, 1207484.2280090372, 1930925.327873265, 1284492.8412734873, 1810826.4571764332, 1489319.4699845063, 1295795.0217862097, 1265590.7092688382, 1770181.1316758918, 1517989.8730817244, 1764688.3580046897, 1029759.6676664538, 983578.476316408, 1016180.8831057044, 1041856.1003330644, 1388387.6622821786, 1740313.839355649, 1889021.743910078, 1155008.4903816702, 1250123.5464966097, 1872980.4648167433, 988079.6106728106, 1075159.0135165844, 1808827.8153116012, 1678428.4087094406, 1168801.1017128346, 1146230.176055767, 2019646.0051988007, 1934669.4155026665, 1022938.0472727956, 1158659.6160124717, 1703619.885379108, 1323221.2715301653, 1800355.8877493602, 1279469.0129666848, 1885772.9433754506, 1756064.092565777, 1771923.942854048, 1410866.7824727867, 1700314.1894324122, 1525190.6540738102, 1046026.8077738517, 1020201.4794473584, 1536910.2317749811, 1563522.746407627, 1514840.055054727, 1012335.5488273496, 1133806.8601449141, 1618034.3556020786, 1710713.756247299, 1288599.0815855328, 1211306.3608443954, 1313329.642058732, 1246101.0027613293, 1114642.3983295537, 1240011.1217967365, 1091047.813314872, 1462724.9052271254, 1015545.2053408388, 968842.2280353209, 1221156.8912736033, 1550521.4898737029, 1784746.8959130724, 1320347.3539813517, 979698.0845743956, 1492420.270554518, 1789441.910930436, 1411084.0184388123, 1908856.4014387464, 1520854.573313904, 1780947.3775773686, 984977.6780926663, 1877756.4949271062, 1933589.566966543, 1637720.2918112641, 1729005.1228877672, 1980773.3464704826, 1367511.7132855807, 1750719.951577734, 1027599.5081649768, 1479172.6340697883, 1618847.2527462824, 1894654.5236308398, 1123919.2566453775, 1887078.650558294, 1733833.951291958, 1335972.3897088056, 1194138.1969796692, 1250997.654512688, 1749588.4354216708, 1005252.1099544087, 1691000.748378443, 1210979.4706598143, 1377924.49030356, 1282334.3031680926, 1309675.7820019017, 1080466.8698593904, 1168728.7065749343, 1587680.173154562, 1107480.9329725862, 1908521.7738618597, 1884791.561609402, 1193137.8792023007, 1583120.6580467618, 1313696.363858461, 1203269.0931349094, 1638101.432365362, 1313223.2694055957, 1717574.6086878679, 1224025.7099400917, 1124707.9379464495, 1291322.3956622991, 1321009.8509675916, 1689307.1093298546, 1137393.9350292843, 1191369.8169692992, 1947007.1755473684, 1950197.8455334671, 1373674.364283764, 1317819.4569345948, 1854890.348350039, 1969352.9838553958, 1699644.8025573771, 1390662.2760400162, 1436873.9304662435, 1799521.1585798373, 1264817.12397481, 1304326.457460092, 1102749.1845267755, 1650729.3831634503, 1702988.2614876705, 1440930.9615944226, 996961.6834719218, 1069730.809100845, 1690679.8648782636, 1582872.8535174404, 1679332.7072337396, 1263404.0878261183, 1990694.3359571537, 1197114.9412438078, 1718919.7828771332, 1993738.6855609578, 991771.9130242608, 1630265.1494180846, 1926256.0928749233, 1895302.7286931225, 1461613.5031051682, 1629810.1116650447, 1562189.45511154, 1111270.4874836302, 1714174.476692882, 1131684.032569456, 1161352.5527352553, 1159204.1382334786, 1865512.079959619, 1400628.0526079107, 1286864.3603891046, 1144257.8710439038, 1343648.1235292882, 1490750.440231418, 1393781.9787455746, 1656080.955352295, 1853848.166834306, 1563298.471525815, 1751784.7034224102, 1489050.6477480035, 1679185.0477632734, 1201832.338023246, 1511276.7679805728, 1405371.5916980489, 1734280.9028710308, 1868223.859781633, 1442752.6331991013, 1043563.130542258, 1223801.940150985, 1950757.6580053444, 1696830.1538170131, 1591475.9697174798, 1567217.2688728226, 1187847.294678079, 1542974.1382953608, 1432295.604659512, 997231.3120657125, 1253606.4004548197, 1719758.379504662, 1168863.3240509415, 1053265.0048448138, 1338443.5665066964, 1536235.4491055447, 1469668.088778234, 1042436.6681092761, 1958906.632428273, 1780152.0108495716, 983968.3795879905, 1059516.8545667338, 1797061.1195363973, 1367246.1360643937, 1499082.6358501993, 1913543.906414636, 2008899.2143945287, 1615923.1692753264, 1588394.7252314985]
### Data Send By All Devices
g1y = [26725.56545975536, 49249.56264578589, 60949.649222614506, 33890.31014468024, 34012.478336592685, 35471.99895301351, 55078.427858205396, 22435.823779457187, 42564.95205136428, 44090.43402319108, 34768.73408743735, 57554.88872761611, 23729.118593110456, 41792.77185601255, 27719.991246497364, 46255.619765400916, 41469.82277922142, 46945.616056693245, 35649.857051963474, 44774.77021987357, 50859.277949830066, 35615.51955577785, 57398.27421028067, 53309.91762715291, 32169.18949481694, 46226.28629948205, 38285.39153036485, 60601.042022567555, 37369.192389414326, 52058.49426873313, 30386.029001856055, 37221.86037160321, 25426.25651363285, 30905.009342486006, 27704.546954949383, 29644.872117919967, 41609.54145094454, 49304.68329568163, 47129.68938501955, 32045.601945347054, 57615.169529174025, 48548.82696367546, 34351.14860873732, 48612.264946573414, 37107.66492899839, 28013.041711757418, 60035.285417062274, 57541.378369443955, 49918.66001658835, 56057.302107420786, 61897.53112270791, 55247.93388542151, 31988.944986759692, 58569.47890688703, 40519.32627987326, 29499.704274631866, 50485.62049587744, 49421.767424271966, 38149.8330999255, 31118.96109734769, 51598.56507627813, 53033.38809421947, 42167.71456505193, 30247.07933292808, 34660.13686450255, 44388.794893408405, 34462.25756448531, 25508.126626662106, 59768.26905086028, 24602.409614998487, 57726.91046584839, 23674.596003402472, 44364.56324047395, 60719.0536659088, 45450.86607873305, 40038.78890244953, 57469.35568462699, 44338.044410804694, 30152.713464423457, 46946.4345518266, 36844.56014206204, 26233.203968298258, 56994.3102167959, 50398.795549284856, 37688.62554051094, 27831.972883013375, 48137.22841922382, 52465.378085753575, 26573.13769986451, 41313.689507600255, 37451.19542267131, 39053.35585117302, 32919.37938094331, 45745.99943718444, 42839.977538182386, 23526.987984960317, 56796.776263780404, 35959.51456039172, 50036.93623948465, 51308.44948773797, 34766.70473508088, 31615.812759837743, 43610.078589146, 48727.4487387648, 47714.12076641914, 63864.97757199464, 31759.072327976228, 62330.26952836507, 35394.23600284646, 27823.772682225797, 54057.67083894924, 48351.003265962776, 29011.322604088204, 30818.50811943428, 50136.4021474518, 50406.07216502959, 47115.682197021306, 59229.1660451828, 36415.56719827846, 30222.851259928684, 45737.07423742889, 39844.170469606666, 44200.501720249966, 47994.80551877092, 53042.759491573124, 54263.3026969971, 44482.820779220034, 22195.398806163525, 37008.6231090846, 46591.85060320228, 54643.613172065816, 21519.724662663804, 30956.389748366084, 55499.97528116106, 58970.526850471375, 62093.34685339027, 46939.71598368627, 44564.094375053835, 47439.90007733617, 49870.54333685719, 41187.91959321276, 63713.378083667456, 34362.285288660445, 45521.434606251925, 33012.609498156664, 27540.34199295883, 28852.329951744116, 48036.10312913116, 44115.72269897218, 42515.36759509333, 42017.31033428246, 29995.360902138786, 37635.37586414943, 50145.291160574845, 30048.59974772985, 35043.65979994262, 36686.1048641184, 41008.95727780793, 32666.658657877037, 34714.923058444285, 39278.792047179726, 29586.621762486157, 46928.82787119027, 44106.90292400467, 55881.47949589503, 24646.72024075001, 43450.14812877243, 32457.027263371314, 25609.627070767776, 56672.58416857507, 36854.70695005688, 63257.028777931846, 55567.19025950811, 37154.610664670356, 36213.43844799969, 60469.689309369285, 28429.299324604373, 45881.778992104395, 29536.200193473916, 47033.95104528391, 35634.260611029145, 40811.03964956348, 34115.54903697961, 33360.988490762065, 52990.42968886248, 58487.47045925071, 34947.75610834094, 38760.795003242114, 55532.02593957751, 24991.500270924804, 41851.274444721545, 48494.21503312236, 41177.1664328316, 25622.772264810228, 51324.101582246, 50554.80229745462, 33191.7550289584, 36003.002158910465, 50037.6593241934, 34254.22277878919, 44014.48157979269, 32517.927809054323, 39325.99182303951, 42605.17849624942, 32932.397566892265, 46019.94857351168, 45691.24821620326, 57753.16043790391, 36370.461376290434, 34925.63318048913, 23932.331443099145, 41209.69896010829, 37898.93031194832, 31151.436502064236, 38651.21926348493, 26391.477685403195, 41839.589423503734, 32279.894576593164, 46244.07940467471, 36470.20741857571, 42187.96082645927, 29813.167198620977, 40399.4117550041, 44464.47269527389, 44798.86586743422, 56851.67915450699, 31795.584130440173, 62345.116786266284, 26721.80288742201, 49030.70664817032, 53012.830617042266, 32791.33198384779, 51739.19788494733, 62595.26244890483, 47642.30168332456, 50621.90226691591, 40762.90482799118, 51623.6914457633, 46416.894348340706, 33506.02169151954, 51869.50886469674, 36379.821933175685, 54741.04917035383, 54845.56807107661, 51329.95691341241, 41000.740658386574, 58723.77188400945, 54775.331537733, 31293.170680202995, 44222.1433147501, 27409.16477554763, 56566.685035441966, 46969.651964185505, 24315.198219093407, 34636.33953978877, 54023.94229303004, 24716.24150275379, 32774.97754074248, 42201.23466878771, 35672.24664496204, 48588.71640006992, 30148.314263377568, 33073.917608349606, 50337.38447519056, 59007.69223809581, 51585.283907562654, 40816.066182897644, 35089.29834365389, 30124.594855177846, 53493.48686354529, 43857.06619654928, 58113.48947183465, 35984.77009621693, 62263.76617608407, 31652.374268449537, 37429.882172975515, 50700.73585670329, 24832.237562464, 30013.933095956767, 45645.60089632789, 43479.68326019931, 61864.20343419932, 55925.342465762835, 29238.689393841923, 31221.600514499525, 47141.638511880665, 33314.18293827939, 47947.38641634539, 24293.6341491888, 27226.529129278766, 26926.865645026995, 40666.45101314006, 24173.388800813187, 29035.501442923432, 49700.80514750265, 49353.63202522033, 50748.255595814975, 26065.16807560809, 35516.40021784788, 45762.51685227257]
### Power Consumed by Devices
g1y = [g1y[i]/g1x[i] for i in range(len(g1x))]
### Power consumed for each unit of data for devices

In [ ]:
plot1=sns.lineplot(x=g1x,y=g1y)
plot1.set(xlabel="Data Send By All Devices",ylabel='Power consumed for each unit of data for devices',ylim = (-0.6,0.6))

## Graph - 2

In [ ]:
def Graph2():
 return (len(obj.deviceData),obj.deviceData.count(0))

In [ ]:
g2y = [387, 293, 294, 368, 184, 198, 349, 159, 239, 221, 379, 317, 293, 265, 366, 369, 399, 261, 300, 216, 366, 337, 339, 253, 231, 185, 384, 270, 289, 169, 307, 306, 214, 234, 285, 366, 223, 272, 257, 162, 273, 234, 224, 363, 339, 302, 306, 321, 351, 295, 347, 183, 303, 327, 250, 384, 241, 169, 182, 163, 397, 198, 358, 376, 386, 219, 175, 239, 351, 256, 162, 241, 343, 203, 165, 392, 257, 193, 246, 385, 152, 319, 236, 353, 354, 372, 360, 270, 359, 278, 301, 165, 273, 211, 318, 369, 351, 376, 316, 153, 260, 305, 349, 275, 373, 273, 329, 181, 165, 264, 303, 194, 245, 223, 376, 225, 263, 354, 376, 246, 279, 306, 190, 259, 233, 244, 205, 250, 240, 173, 320, 150, 385, 311, 206, 227, 255, 354, 384, 317, 249, 250, 365, 270, 385, 368, 257, 276, 204, 318, 384, 171, 352, 399, 155, 232, 161, 176, 150, 366, 398, 180, 177, 249, 370, 391, 357, 288, 250, 241, 342, 357, 394, 177, 309, 366, 370, 342, 207, 194, 188, 333, 318, 151, 150, 218, 153, 340, 345, 363, 201, 319, 308, 153, 193, 393, 300, 182, 198, 167, 235, 227, 269, 279, 309, 322, 168, 331, 344, 382, 160, 386, 230, 384, 245, 311, 362, 213, 278, 189, 336, 286, 156, 213, 350, 306, 332, 256, 397, 266, 368, 251, 176, 397, 286, 340, 156, 240, 170, 289, 251, 317, 381, 189, 370, 169, 311, 304, 340, 355, 331, 271, 317, 179, 360, 398, 247, 270, 342, 236, 298, 217, 310, 170, 280, 162, 291, 252, 303, 156, 306, 326, 159, 153, 273, 278, 396, 370, 185, 207, 263, 268, 155, 263, 237, 329, 185, 332, 184, 268, 367, 358, 252, 296, 320, 269, 348, 230, 272, 169]                    
### Total no of satisfied devices
g2x = [344, 270, 275, 336, 169, 182, 325, 145, 224, 195, 349, 288, 268, 236, 325, 334, 366, 242, 275, 198, 331, 303, 309, 230, 211, 165, 350, 243, 254, 155, 281, 268, 186, 210, 260, 333, 196, 249, 235, 154, 252, 212, 208, 343, 313, 280, 279, 296, 304, 269, 320, 162, 269, 296, 214, 342, 218, 153, 169, 154, 365, 182, 324, 350, 349, 198, 149, 213, 318, 228, 143, 219, 311, 184, 145, 351, 221, 178, 223, 355, 134, 294, 217, 311, 323, 346, 324, 249, 324, 264, 273, 148, 241, 184, 291, 330, 315, 352, 289, 141, 236, 281, 325, 253, 340, 251, 294, 168, 150, 231, 274, 177, 222, 201, 335, 207, 240, 323, 345, 224, 257, 266, 166, 230, 206, 221, 191, 224, 221, 151, 285, 135, 349, 287, 191, 210, 235, 317, 349, 285, 223, 230, 346, 244, 353, 343, 228, 257, 182, 285, 347, 152, 313, 366, 140, 210, 148, 150, 140, 324, 363, 158, 162, 231, 340, 356, 332, 262, 231, 216, 316, 318, 355, 161, 275, 329, 323, 312, 184, 164, 176, 301, 288, 141, 134, 191, 137, 306, 313, 328, 185, 294, 277, 136, 170, 349, 264, 171, 172, 153, 217, 205, 247, 258, 292, 289, 156, 297, 299, 358, 140, 344, 208, 356, 215, 278, 329, 193, 260, 177, 302, 261, 141, 195, 300, 264, 303, 230, 346, 240, 336, 227, 161, 355, 257, 316, 138, 209, 159, 263, 221, 295, 345, 165, 332, 156, 289, 273, 312, 327, 306, 251, 277, 161, 315, 364, 219, 243, 310, 213, 269, 188, 274, 157, 254, 147, 262, 231, 283, 144, 284, 292, 150, 140, 251, 251, 362, 338, 167, 181, 235, 254, 143, 239, 210, 293, 168, 303, 164, 250, 329, 320, 229, 264, 283, 244, 316, 207, 247, 149]     
### Total no of devices
plot2=sns.lineplot(y=g2x,x=g2y)
plot2.set(ylabel='No. of satisfied devices',xlabel='Total no of devices')

##Graph 3

In [ ]:
def Graph3():
  dataSentByDevices = np.subtract(obj.ECMaxStorage, obj.ECStorage)
  

  dataSentBySatisfiedDevices = 0
  for i in range(n):
    if obj.deviceData[i] == 0:
      dataSentBySatisfiedDevices += AllocationData[i]

  return (sum(dataSentByDevices),dataSentBySatisfiedDevices)


In [ ]:
g3x = [1766357.8132679758, 1537572.9824533144, 1747055.6188805948, 1057855.2142292093, 1012463.190920355, 1779876.0513704885, 1665026.5889087839, 1733775.3060132693, 1830261.6292181802, 1733286.1421435354, 1928505.715245211, 1158185.5762530232, 1821320.0302981401, 1254921.4499200638, 1442657.3670760782, 1911030.4027949984, 1510769.9298335137, 1180359.8279166603, 1857396.3250698466, 1924031.9280750523, 1378846.8216817447, 1691835.3128196457, 1520190.0982180014, 1497378.7536213421, 1278716.0743449207, 1900617.976176619, 1402989.9470415905, 1874931.2871586136, 1027689.4423867201, 1299515.3603396094, 1096517.0525017856, 1935191.072141861, 1709627.9842155068, 1119826.8199217205, 1215165.9847202138, 1250519.4854492843, 1065240.2512118458, 1043050.7245134466, 1286456.5712690665, 1155492.7616311302, 1252739.7917654512, 1175104.535945504, 1146420.4482521429, 1052575.4545894102, 1396510.9857859584, 1640431.4801805625, 1345025.8045618285, 1584478.974603421, 1115733.1438590274, 1578068.089498487, 1734486.5533344324, 1363701.6641321415, 1509259.0253042195, 1432826.9449693155, 1913452.4425098838, 1496547.610730024, 1126675.8465190586, 1311169.457859443, 1229837.8309263377, 1422516.1677278704, 1131330.774225748, 1711453.2547979003, 1551450.2079009286, 1135421.8745096885, 1108607.244761684, 1602129.053199319, 1541681.2510060086, 1326264.1462642776, 1825406.6434300814, 1987069.3082223616, 1369178.992406445, 1505492.5351470243, 1459389.8964827019, 1983612.1021482951, 1882856.8722848746, 1504209.6346939767, 1092242.0440395079, 1936106.734119744, 1880043.7440686063, 1856813.2754515237, 1118415.7569940647, 1052255.1637912935, 1841968.2779959384, 1919617.3030819688, 1749441.2225518245, 1271245.5709352866, 1016332.2078352435, 1451153.058943783, 1575697.4795142892, 1435863.5153869656, 1262896.3864885499, 1391464.1574297012, 1167639.95825389, 1023798.6262804937, 1841194.257251824, 1989425.7359981367, 1977219.6782805556, 1528763.0305654649, 1148853.4703715297, 1207484.2280090372, 1930925.327873265, 1284492.8412734873, 1810826.4571764332, 1489319.4699845063, 1295795.0217862097, 1265590.7092688382, 1770181.1316758918, 1517989.8730817244, 1764688.3580046897, 1029759.6676664538, 983578.476316408, 1016180.8831057044, 1041856.1003330644, 1388387.6622821786, 1740313.839355649, 1889021.743910078, 1155008.4903816702, 1250123.5464966097, 1872980.4648167433, 988079.6106728106, 1075159.0135165844, 1808827.8153116012, 1678428.4087094406, 1168801.1017128346, 1146230.176055767, 2019646.0051988007, 1934669.4155026665, 1022938.0472727956, 1158659.6160124717, 1703619.885379108, 1323221.2715301653, 1800355.8877493602, 1279469.0129666848, 1885772.9433754506, 1756064.092565777, 1771923.942854048, 1410866.7824727867, 1700314.1894324122, 1525190.6540738102, 1046026.8077738517, 1020201.4794473584, 1536910.2317749811, 1563522.746407627, 1514840.055054727, 1012335.5488273496, 1133806.8601449141, 1618034.3556020786, 1710713.756247299, 1288599.0815855328, 1211306.3608443954, 1313329.642058732, 1246101.0027613293, 1114642.3983295537, 1240011.1217967365, 1091047.813314872, 1462724.9052271254, 1015545.2053408388, 968842.2280353209, 1221156.8912736033, 1550521.4898737029, 1784746.8959130724, 1320347.3539813517, 979698.0845743956, 1492420.270554518, 1789441.910930436, 1411084.0184388123, 1908856.4014387464, 1520854.573313904, 1780947.3775773686, 984977.6780926663, 1877756.4949271062, 1933589.566966543, 1637720.2918112641, 1729005.1228877672, 1980773.3464704826, 1367511.7132855807, 1750719.951577734, 1027599.5081649768, 1479172.6340697883, 1618847.2527462824, 1894654.5236308398, 1123919.2566453775, 1887078.650558294, 1733833.951291958, 1335972.3897088056, 1194138.1969796692, 1250997.654512688, 1749588.4354216708, 1005252.1099544087, 1691000.748378443, 1210979.4706598143, 1377924.49030356, 1282334.3031680926, 1309675.7820019017, 1080466.8698593904, 1168728.7065749343, 1587680.173154562, 1107480.9329725862, 1908521.7738618597, 1884791.561609402, 1193137.8792023007, 1583120.6580467618, 1313696.363858461, 1203269.0931349094, 1638101.432365362, 1313223.2694055957, 1717574.6086878679, 1224025.7099400917, 1124707.9379464495, 1291322.3956622991, 1321009.8509675916, 1689307.1093298546, 1137393.9350292843, 1191369.8169692992, 1947007.1755473684, 1950197.8455334671, 1373674.364283764, 1317819.4569345948, 1854890.348350039, 1969352.9838553958, 1699644.8025573771, 1390662.2760400162, 1436873.9304662435, 1799521.1585798373, 1264817.12397481, 1304326.457460092, 1102749.1845267755, 1650729.3831634503, 1702988.2614876705, 1440930.9615944226, 996961.6834719218, 1069730.809100845, 1690679.8648782636, 1582872.8535174404, 1679332.7072337396, 1263404.0878261183, 1990694.3359571537, 1197114.9412438078, 1718919.7828771332, 1993738.6855609578, 991771.9130242608, 1630265.1494180846, 1926256.0928749233, 1895302.7286931225, 1461613.5031051682, 1629810.1116650447, 1562189.45511154, 1111270.4874836302, 1714174.476692882, 1131684.032569456, 1161352.5527352553, 1159204.1382334786, 1865512.079959619, 1400628.0526079107, 1286864.3603891046, 1144257.8710439038, 1343648.1235292882, 1490750.440231418, 1393781.9787455746, 1656080.955352295, 1853848.166834306, 1563298.471525815, 1751784.7034224102, 1489050.6477480035, 1679185.0477632734, 1201832.338023246, 1511276.7679805728, 1405371.5916980489, 1734280.9028710308, 1868223.859781633, 1442752.6331991013, 1043563.130542258, 1223801.940150985, 1950757.6580053444, 1696830.1538170131, 1591475.9697174798, 1567217.2688728226, 1187847.294678079, 1542974.1382953608, 1432295.604659512, 997231.3120657125, 1253606.4004548197, 1719758.379504662, 1168863.3240509415, 1053265.0048448138, 1338443.5665066964, 1536235.4491055447, 1469668.088778234, 1042436.6681092761, 1958906.632428273, 1780152.0108495716, 983968.3795879905, 1059516.8545667338, 1797061.1195363973, 1367246.1360643937, 1499082.6358501993, 1913543.906414636, 2008899.2143945287, 1615923.1692753264, 1588394.7252314985]      
#Data sent by all devices
g3y = [8163, 15915, 18944, 9957, 10475, 11741, 17617, 6927, 14661, 15186, 12469, 18060, 7731, 12855, 8064, 15725, 12484, 14948, 11644, 13732, 16199, 11866, 17756, 17187, 9748, 13698, 12122, 18137, 11078, 16330, 9404, 11735, 7625, 9048, 8577, 9706, 13350, 16731, 14654, 9610, 19461, 16755, 11437, 14867, 11235, 8992, 19919, 18572, 16053, 17860, 20179, 19922, 10568, 18066, 12682, 9314, 18647, 15337, 12425, 9046, 18849, 18474, 12504, 8745, 10755, 13216, 10656, 7307, 17760, 7469, 16850, 7228, 14626, 18135, 14153, 12401, 17805, 13881, 9166, 14952, 11672, 7920, 18003, 17006, 11343, 9021, 15276, 18854, 8741, 12994, 13057, 11398, 9883, 16095, 12723, 7608, 17661, 11329, 16375, 16335, 9957, 9552, 14851, 14684, 15244, 19093, 9845, 18621, 10603, 8065, 18264, 17500, 8405, 9669, 15060, 15023, 14185, 17606, 11692, 9442, 15765, 11243, 14263, 14721, 15971, 16677, 14898, 7276, 11121, 15342, 17056, 7089, 10204, 17655, 18534, 18096, 14728, 13574, 14851, 15427, 13739, 18350, 11688, 15510, 10385, 7607, 9536, 14630, 14736, 13187, 12862, 9196, 11525, 15394, 9019, 11631, 11345, 14111, 10826, 10256, 12063, 9386, 13967, 12595, 17313, 7557, 13274, 10148, 8092, 18767, 10904, 19285, 16479, 12075, 11719, 19075, 8773, 15298, 10266, 13709, 10151, 12859, 10140, 10104, 18660, 17509, 11281, 11016, 17112, 7687, 13358, 16737, 13331, 8404, 15233, 17528, 10382, 10915, 15448, 10642, 15650, 10750, 11973, 13102, 9629, 13396, 16107, 17724, 11219, 11207, 7744, 13281, 11023, 9784, 11848, 7209, 12808, 10099, 14494, 10915, 12575, 8642, 12552, 13628, 14812, 19866, 10555, 18732, 8726, 16231, 16236, 9919, 16478, 18565, 14019, 14226, 12525, 17398, 14318, 9995, 15657, 10755, 18265, 16849, 16002, 12221, 17586, 19317, 9556, 13679, 8163, 17393, 13538, 7520, 10633, 18267, 7368, 9634, 14281, 10878, 15297, 8989, 10198, 16952, 19153, 18000, 12900, 12157, 9578, 17717, 13742, 18830, 11803, 18976, 9126, 13095, 16508, 8248, 9796, 15826, 13206, 19299, 17293, 8710, 9692, 17751, 10556, 17309, 7231, 9245, 7505, 12532, 7900, 9105, 15946, 18303, 17935, 8153, 10875, 14379]
#Data sent by satisfied devices
g3y = [g3y[i]/g3x[i] for i in range(len(g3x))]
##Data sent by satisfied devices for 1 unit of total data sent by devices
plot3=sns.lineplot(x=g3x,y=g3y)
plot3.set(xlabel='Data sent by all devices',ylabel='Data sent by satisfied devices for 1 unit of \ntotal data sent by devices',ylim = (-0.6,0.6))

##Graph 4

In [ ]:
def Graph4():
  powerConsumedByDevices = np.subtract(obj.deviceMaxBattery, obj.deviceBattery)

  powerConsumedByECs = 0
  for i in range(m):
    if ECStorage[i]>0:
      powerConsumedByECs += ECPower[i]

  return (sum(powerConsumedByDevices), powerConsumedByECs)

In [ ]:
g4x = [26725.56545975536, 49249.56264578589, 60949.649222614506, 33890.31014468024, 34012.478336592685, 35471.99895301351, 55078.427858205396, 22435.823779457187, 42564.95205136428, 44090.43402319108, 34768.73408743735, 57554.88872761611, 23729.118593110456, 41792.77185601255, 27719.991246497364, 46255.619765400916, 41469.82277922142, 46945.616056693245, 35649.857051963474, 44774.77021987357, 50859.277949830066, 35615.51955577785, 57398.27421028067, 53309.91762715291, 32169.18949481694, 46226.28629948205, 38285.39153036485, 60601.042022567555, 37369.192389414326, 52058.49426873313, 30386.029001856055, 37221.86037160321, 25426.25651363285, 30905.009342486006, 27704.546954949383, 29644.872117919967, 41609.54145094454, 49304.68329568163, 47129.68938501955, 32045.601945347054, 57615.169529174025, 48548.82696367546, 34351.14860873732, 48612.264946573414, 37107.66492899839, 28013.041711757418, 60035.285417062274, 57541.378369443955, 49918.66001658835, 56057.302107420786, 61897.53112270791, 55247.93388542151, 31988.944986759692, 58569.47890688703, 40519.32627987326, 29499.704274631866, 50485.62049587744, 49421.767424271966, 38149.8330999255, 31118.96109734769, 51598.56507627813, 53033.38809421947, 42167.71456505193, 30247.07933292808, 34660.13686450255, 44388.794893408405, 34462.25756448531, 25508.126626662106, 59768.26905086028, 24602.409614998487, 57726.91046584839, 23674.596003402472, 44364.56324047395, 60719.0536659088, 45450.86607873305, 40038.78890244953, 57469.35568462699, 44338.044410804694, 30152.713464423457, 46946.4345518266, 36844.56014206204, 26233.203968298258, 56994.3102167959, 50398.795549284856, 37688.62554051094, 27831.972883013375, 48137.22841922382, 52465.378085753575, 26573.13769986451, 41313.689507600255, 37451.19542267131, 39053.35585117302, 32919.37938094331, 45745.99943718444, 42839.977538182386, 23526.987984960317, 56796.776263780404, 35959.51456039172, 50036.93623948465, 51308.44948773797, 34766.70473508088, 31615.812759837743, 43610.078589146, 48727.4487387648, 47714.12076641914, 63864.97757199464, 31759.072327976228, 62330.26952836507, 35394.23600284646, 27823.772682225797, 54057.67083894924, 48351.003265962776, 29011.322604088204, 30818.50811943428, 50136.4021474518, 50406.07216502959, 47115.682197021306, 59229.1660451828, 36415.56719827846, 30222.851259928684, 45737.07423742889, 39844.170469606666, 44200.501720249966, 47994.80551877092, 53042.759491573124, 54263.3026969971, 44482.820779220034, 22195.398806163525, 37008.6231090846, 46591.85060320228, 54643.613172065816, 21519.724662663804, 30956.389748366084, 55499.97528116106, 58970.526850471375, 62093.34685339027, 46939.71598368627, 44564.094375053835, 47439.90007733617, 49870.54333685719, 41187.91959321276, 63713.378083667456, 34362.285288660445, 45521.434606251925, 33012.609498156664, 27540.34199295883, 28852.329951744116, 48036.10312913116, 44115.72269897218, 42515.36759509333, 42017.31033428246, 29995.360902138786, 37635.37586414943, 50145.291160574845, 30048.59974772985, 35043.65979994262, 36686.1048641184, 41008.95727780793, 32666.658657877037, 34714.923058444285, 39278.792047179726, 29586.621762486157, 46928.82787119027, 44106.90292400467, 55881.47949589503, 24646.72024075001, 43450.14812877243, 32457.027263371314, 25609.627070767776, 56672.58416857507, 36854.70695005688, 63257.028777931846, 55567.19025950811, 37154.610664670356, 36213.43844799969, 60469.689309369285, 28429.299324604373, 45881.778992104395, 29536.200193473916, 47033.95104528391, 35634.260611029145, 40811.03964956348, 34115.54903697961, 33360.988490762065, 52990.42968886248, 58487.47045925071, 34947.75610834094, 38760.795003242114, 55532.02593957751, 24991.500270924804, 41851.274444721545, 48494.21503312236, 41177.1664328316, 25622.772264810228, 51324.101582246, 50554.80229745462, 33191.7550289584, 36003.002158910465, 50037.6593241934, 34254.22277878919, 44014.48157979269, 32517.927809054323, 39325.99182303951, 42605.17849624942, 32932.397566892265, 46019.94857351168, 45691.24821620326, 57753.16043790391, 36370.461376290434, 34925.63318048913, 23932.331443099145, 41209.69896010829, 37898.93031194832, 31151.436502064236, 38651.21926348493, 26391.477685403195, 41839.589423503734, 32279.894576593164, 46244.07940467471, 36470.20741857571, 42187.96082645927, 29813.167198620977, 40399.4117550041, 44464.47269527389, 44798.86586743422, 56851.67915450699, 31795.584130440173, 62345.116786266284, 26721.80288742201, 49030.70664817032, 53012.830617042266, 32791.33198384779, 51739.19788494733, 62595.26244890483, 47642.30168332456, 50621.90226691591, 40762.90482799118, 51623.6914457633, 46416.894348340706, 33506.02169151954, 51869.50886469674, 36379.821933175685, 54741.04917035383, 54845.56807107661, 51329.95691341241, 41000.740658386574, 58723.77188400945, 54775.331537733, 31293.170680202995, 44222.1433147501, 27409.16477554763, 56566.685035441966, 46969.651964185505, 24315.198219093407, 34636.33953978877, 54023.94229303004, 24716.24150275379, 32774.97754074248, 42201.23466878771, 35672.24664496204, 48588.71640006992, 30148.314263377568, 33073.917608349606, 50337.38447519056, 59007.69223809581, 51585.283907562654, 40816.066182897644, 35089.29834365389, 30124.594855177846, 53493.48686354529, 43857.06619654928, 58113.48947183465, 35984.77009621693, 62263.76617608407, 31652.374268449537, 37429.882172975515, 50700.73585670329, 24832.237562464, 30013.933095956767, 45645.60089632789, 43479.68326019931, 61864.20343419932, 55925.342465762835, 29238.689393841923, 31221.600514499525, 47141.638511880665, 33314.18293827939, 47947.38641634539, 24293.6341491888, 27226.529129278766, 26926.865645026995, 40666.45101314006, 24173.388800813187, 29035.501442923432, 49700.80514750265, 49353.63202522033, 50748.255595814975, 26065.16807560809, 35516.40021784788, 45762.51685227257]
#Power consumed by devices
g4y = [48492, 84746, 63911, 38928, 75830, 45191, 80236, 36030, 46757, 62279, 38236, 97438, 39965, 108060, 46979, 59413, 60015, 65414, 61824, 57878, 81533, 23501, 53378, 59064, 64836, 47641, 70268, 75426, 62029, 77716, 49103, 53904, 36773, 51209, 48109, 66755, 65974, 92368, 89136, 54246, 73565, 61421, 26908, 77573, 56043, 30351, 87971, 81705, 86250, 89756, 68895, 78083, 66281, 87867, 61126, 45440, 95613, 58782, 41935, 40153, 86768, 64167, 51535, 30752, 44857, 80011, 50549, 73569, 76366, 32509, 52791, 43137, 61189, 94068, 45558, 67938, 69894, 62964, 59668, 59574, 40701, 36242, 67540, 84875, 92250, 44513, 78542, 78865, 34325, 47211, 65431, 52144, 58922, 84579, 60883, 35688, 72394, 54212, 83720, 65746, 40831, 50260, 52121, 58419, 87396, 69887, 38761, 61376, 51464, 70585, 104614, 85054, 43690, 44696, 54016, 64798, 65057, 102978, 29507, 93735, 66952, 50483, 95788, 48948, 66473, 46342, 37336, 53041, 66103, 72530, 75803, 33688, 53184, 51733, 50939, 68875, 84141, 57110, 60246, 43232, 80823, 69806, 46298, 58548, 36168, 39848, 35682, 70761, 77820, 66956, 66078, 65792, 67625, 55340, 74923, 74534, 66582, 52121, 48194, 49113, 78609, 54119, 75732, 59395, 85990, 42684, 65098, 39127, 42126, 64073, 59864, 50775, 103547, 59086, 51499, 72297, 57689, 52801, 46785, 50358, 60862, 66021, 53998, 43321, 87163, 79676, 40551, 59784, 77327, 66406, 45252, 90244, 78555, 44002, 85335, 115438, 62187, 58234, 69025, 52686, 81752, 36360, 30474, 69325, 41306, 63964, 58500, 57937, 51363, 73791, 51682, 40681, 66842, 75469, 59852, 36170, 73918, 56782, 85719, 36885, 62620, 40767, 55094, 80112, 72323, 97997, 65179, 74521, 42109, 58791, 83842, 41336, 65802, 76116, 62687, 78978, 57504, 64549, 46869, 28211, 83608, 47799, 60298, 63592, 65177, 52520, 103484, 97781, 33340, 39877, 34107, 61575, 53480, 41713, 43528, 90474, 37169, 42932, 76632, 49271, 99148, 42166, 46728, 60716, 78065, 79544, 59915, 46497, 58342, 70857, 79162, 100251, 42026, 95099, 74520, 71598, 65963, 36849, 44442, 77597, 66865, 71878, 81436, 58593, 46720, 86453, 60762, 82370, 38882, 58743, 34944, 62276, 35231, 37969, 67772, 86247, 62937, 33580, 46160, 84714]
#Power consumed by EC 
plot4=sns.lineplot(x=g4x,y=g4y)
plot4.set(xlabel='Power consumed by devices',ylabel='Power consumed by EC')

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx.drawing.layout as lyts

devices = ['d0', 'd1', 'd2', 'd3', 'd4', 'd5']
ecomp = ['e0', 'e1', 'e2', 'e3']

E = (('d0', 'e0', 12), ('d1', 'e0', 15), ('d0', 'e1', 12), ('d2', 'e1', 13), ('d1', 'e2', 15), ('d3', 'e2', 12), ('d4', 'e2', 19), ('d2', 'e3', 13), ('d4', 'e3', 19), ('d5', 'e3', 20))

G = nx.DiGraph()

G.add_nodes_from(ecomp, biparite=0)
G.add_nodes_from(devices, biparite=2)

G.add_weighted_edges_from(E)

pos = nx.bipartite_layout(G, devices, align='horizontal')

weight = nx.get_edge_attributes(G, 'weight')

color_map = []
for node in G:
    if node in devices:
        color_map.append('blue')
    else: 
        color_map.append('red') 

plt.figure(3,figsize=(15,7))

nx.draw_networkx(G, pos, node_size= 1000, edge_color='black', node_color=color_map, arrowsize=15, font_size=14, font_color='white', with_labels=True)

nx.draw_networkx_edge_labels(G, pos, weight)

# ax = plt.gca()
# ax.margins(0.08)
plt.axis("off")
# plt.tight_layout()
plt.show()

In [ ]:
deviceMaxBattery=[150,80,130,125,98]
deviceBattery=deviceMaxBattery[::]
deviceData=[20,30,25,30,24]

ECPower=[20,55,49,39]

ECStorage=[0,0,0,0]
ECMaxStorage=[35,45,35,40]
inf=float('inf')

In [ ]:


##device-edge-power
inputArray=[[0, 0, 4.0],  
 [0, 1, 4.0],
 [1, 0, 6.0],
 [1, 2, 6.0],
 [2, 0, 5.0],
 [2, 1, 5.0],
 [3, 2, 6.0],
 [3, 3, 6.0],
 [4, 3, 4.8]]


##NEW GRAPH

In [ ]:
noOfValues = 300
ECCount = 10
deviceCount = []
totalPower = []

for iii in range(noOfValues):
  m = ECCount ##No of edge devices
  n = random.randint(minDevices,maxDevices) ##No of devices

  deviceMaxBattery,deviceData,ECPower,ECMaxStorage,ECStorage = [],[],[],[],[0 for i in range(m)]

  for i in range(m):
    ECPower.append(random.randint(minEdgePower,maxEdgePower))
    ECMaxStorage.append(random.randint(minEdgeStorage,maxEdgeStorage))

  for i in range(n):
    deviceMaxBattery.append(random.uniform(minDeviceBattery,maxDeviceBattery))
    deviceData.append(random.randint(minDeviceStorage,maxDeviceStorage))
  deviceBattery=deviceMaxBattery[::]
  AllocationData = deviceData[:]
  inputArray = [] ##device-edge-power
  for i in range(n):
    numberOfEdges = random.randint(1,m//2+1)
    connectedEdges = random.sample(range(m),numberOfEdges)
    for j in connectedEdges:
      inputArray.append([i,j,float(random.randint(minTransmissionPower,maxTransmissionPower))])
  obj = EdgeEnvironment(deviceBattery,deviceMaxBattery,ECPower,ECStorage,ECMaxStorage,deviceData)
  obj.extractInput(inputArray)  
  AllocationData = deviceData[:]
  obj.startAllocation()
  deviceCount.append(n)
  power = 0
  for i in range(n):power += deviceMaxBattery[i]-deviceBattery[i]
  for i in range(m):
    if ECStorage[i]:power+=ECPower[i]
  totalPower.append(power)

  print(iii+1,end=' ')

print()


In [ ]:
print(deviceCount)
print(totalPower)

In [ ]:
##ECCount = 20
deviceCount20 = [286, 310, 193, 219, 158, 336, 157, 153, 371, 260, 327, 229, 327, 260, 372, 285, 175, 316, 322, 199, 168, 328, 164, 257, 259, 300, 224, 190, 273, 308, 226, 400, 304, 292, 192, 298, 267, 257, 352, 334, 384, 375, 324, 267, 280, 206, 211, 337, 322, 374, 351, 228, 317, 177, 364, 201, 337, 363, 234, 352, 209, 320, 191, 251, 265, 249, 195, 161, 295, 296, 252, 313, 267, 380, 177, 222, 283, 338, 215, 260, 270, 295, 216, 248, 399, 305, 226, 238, 399, 194, 374, 296, 196, 178, 223, 286, 189, 371, 276, 206, 369, 268, 364, 234, 375, 171, 321, 210, 235, 373, 350, 301, 322, 327, 180, 392, 280, 242, 327, 258, 178, 298, 262, 358, 201, 152, 384, 238, 322, 216, 309, 170, 385, 278, 165, 247, 226, 370, 343, 309, 379, 322, 298, 237, 383, 397, 376, 269, 333, 332, 213, 304, 181, 218, 155, 169, 202, 352, 298, 273, 223, 393, 222, 362, 235, 313, 294, 343, 221, 193, 362, 267, 266, 153, 214, 282, 207, 252, 375, 237, 303, 321, 220, 354, 170, 163, 284, 364, 219, 268, 382, 241, 245, 321, 351, 271, 177, 335, 294, 178, 340, 218, 261, 219, 228, 285, 153, 206, 364, 200, 151, 279, 206, 307, 345, 265, 394, 250, 291, 150, 208, 364, 196, 233, 212, 268, 198, 263, 193, 334, 238, 315, 398, 210, 208, 170, 272, 200, 393, 246, 221, 259, 271, 374, 377, 254, 172, 265, 291, 314, 359, 299, 380, 224, 184, 178, 295, 177, 269, 296, 199, 368, 225, 256, 294, 331, 179, 390, 227, 217, 273, 371, 227, 382, 363, 274, 394, 309, 364, 160, 361, 369, 164, 230, 344, 347, 347, 295, 281, 332, 173, 161, 372, 337, 318, 215, 192, 259, 324, 220]
totalPower20 = [147767.9001140981, 103858.627226431, 104972.48225986937, 126889.05506089685, 108423.29850250241, 123365.18372388971, 91542.67580182305, 86622.11072239124, 127775.4888724036, 120425.09520855089, 137782.18387612293, 127241.35184173379, 131225.56403411983, 132947.43869295326, 130580.92818228445, 128016.13046218347, 101286.5657697391, 122899.76817840585, 146320.7691504435, 105861.35794477622, 84142.71302160286, 134666.70489868964, 80021.0, 105182.30532242118, 112906.07112440324, 119336.50801998614, 130868.49417441695, 113443.69921026277, 144009.4496891091, 146828.67942455388, 112315.0, 162606.5624839594, 114282.26195777585, 108060.40673492773, 71288.48964806469, 134290.98290948186, 142402.7248043567, 117392.7741882554, 153403.9440625433, 123508.32054888364, 138244.1257913466, 148961.83060789417, 152543.86809757928, 116280.4309279675, 96360.97692828174, 96802.24681996547, 130980.49323290591, 123858.48805533764, 121054.20044509688, 141197.02962685848, 137764.9452317049, 114905.71193524488, 144899.60462185807, 99854.20604138217, 141676.19669653525, 114931.06605705497, 150338.87235263595, 135800.38717146436, 122007.57041813634, 141310.54576170296, 110258.48536110055, 108888.02035909699, 99293.11438545589, 110285.0, 144096.94520044603, 93229.7443462369, 90652.0, 102469.01997795582, 125962.9592662582, 125107.45659383338, 122494.7572095556, 98480.27775853156, 142153.00424368534, 159659.35875820485, 94655.96577371166, 123121.28664957642, 112993.22988355107, 129157.88053319837, 115811.1719774033, 106415.5970573211, 118190.52159993403, 143025.44706013784, 93401.9132914717, 97593.13281402682, 132457.19902245764, 128711.47082965495, 118704.28048518903, 103126.29336425498, 149943.56442645512, 104904.0054202344, 148242.15271833056, 129999.0, 115716.29876653176, 114430.39246670227, 121663.63289269873, 124584.30782524307, 96722.15593250381, 127845.39854787568, 142908.29462056083, 102803.96001844891, 150256.96317198576, 119592.12129227041, 121700.58495573586, 119428.32503737116, 137653.11792967492, 139282.36229962646, 126687.51719227034, 96093.83226879794, 108722.19455511811, 125431.00330152504, 135603.82698336348, 138324.0990936137, 134737.23932539794, 124760.07948996697, 101699.73417640934, 115080.2411652515, 110202.20103554359, 123415.63331116855, 129966.84736294272, 125330.3243871512, 96773.14152883344, 117409.11879396177, 125869.49129974253, 136817.8628380553, 93655.69221026078, 112936.13778627125, 137097.87656782696, 117407.41242309281, 137893.54073680966, 109448.11770275723, 140268.53840812284, 86323.0, 145682.533988578, 124428.28187507429, 108201.50817961863, 137873.67138959063, 112344.21387044413, 153464.92243381718, 137928.75123100233, 121620.31511614224, 150399.51185276773, 128923.68006002574, 135622.76234565902, 111141.02541135249, 143846.53906773898, 147646.39581262256, 150524.38602664595, 119118.89950005291, 144088.28547776345, 132419.1132488891, 113199.46417418247, 126357.69645436511, 100563.7307909107, 117423.302437455, 91596.95146872546, 77909.1105336167, 98792.1868680313, 144422.9711535034, 124036.66978371661, 142681.613705519, 114610.49375982223, 121371.88149007647, 127925.47203462667, 119774.91063981972, 150991.93714202312, 123062.23336480578, 128740.62579899536, 144541.00834869567, 129440.35138651941, 95025.42003140948, 150159.25597817492, 127890.90809449743, 130439.90339820806, 95319.59969269135, 112919.89595255622, 132042.9282847482, 111839.62755525962, 118568.40498986377, 153010.44346488232, 117473.39159010115, 133251.8788312434, 139310.1208708451, 137470.11713475804, 132555.86043110496, 98776.6061800831, 89691.23947201474, 134398.80393260723, 118311.46936192727, 96194.53662789661, 128587.3197983893, 138258.2758040765, 131721.4893027273, 111707.36476134026, 120292.51578506535, 139189.39223824325, 128056.45653162048, 105094.07320598904, 156144.8317639177, 122194.69806465058, 116673.04504357214, 136566.77371612494, 104158.90877899039, 111249.14277455898, 106747.51144137484, 116663.0, 124983.47146828602, 130165.72294139891, 75932.27311697183, 129309.71933179171, 98861.6621556836, 96624.48270798862, 114230.55318075749, 110178.32064858367, 124149.93248809496, 158080.2320066623, 131362.82739440142, 158972.69515715673, 116916.26203868716, 105211.46355938725, 100774.03725090186, 111007.49271935926, 151994.13637621872, 123853.45701358149, 131985.19002902415, 105495.91043071642, 106764.53913126906, 98914.0, 111345.98475381633, 128949.10695919365, 140817.8505853481, 122402.4874077646, 165189.6969588106, 115152.45577839913, 108572.59205553953, 103139.10266973302, 114224.6471467957, 108931.81028077686, 111723.94741940184, 135878.882961735, 135546.9445784016, 98251.38986568319, 115221.54080336704, 103105.2722972809, 139498.52363653813, 158684.5860576983, 112759.53132691645, 106689.53593947724, 132047.70883278345, 106016.971401499, 132449.99570725675, 153097.624619274, 133764.3351814506, 157709.80021026082, 109556.21087890386, 126400.60302085784, 106770.43517978973, 137888.71375345608, 90111.3312188419, 117408.0490188523, 137415.0, 92915.76866259203, 160113.65005693017, 114211.12475597767, 121287.37926531406, 132813.59476271027, 150617.10309088824, 95131.16375579205, 153894.3572542067, 126377.994956894, 97196.9978101293, 116339.14848899705, 164125.15887174665, 124705.48070112515, 152927.33613123494, 146360.48741712645, 115807.4539951615, 132084.8342480213, 127563.99877280915, 149252.2720022181, 114911.72571292726, 146426.72887919436, 138247.04454816686, 108910.38337065141, 121594.07452252705, 144902.6207081778, 155445.98835180898, 172484.14868208708, 127649.70659565303, 135024.36611388222, 115837.12606969429, 91732.0, 90057.25443612676, 154119.15042052095, 148692.62010788434, 140264.54670337006, 117556.5182248677, 123963.19893333924, 140225.95846351297, 118802.98379151168, 96154.33831169517]

plot4=sns.lineplot(x=deviceCount,y=totalPower)
plot4.set(xlabel='no of devices',ylabel='total Power')